# 代码测试

## 按照汉字拼音排序


In [ ]:
import locale
from functools import cmp_to_key
import pandas as pd

In [ ]:
locale.setlocale(locale.LC_COLLATE, 'zh_CN.utf8')

In [ ]:
str_list: list[str] = ['硫', '氨', '氯', '碳', '氧', '氯化氢']
str_list

In [ ]:
a = sorted(str_list, key=cmp_to_key(locale.strcoll))
a

In [ ]:
sorted(str_list, key=lambda x: x.encode('gbk'))

In [ ]:
df = pd.DataFrame(str_list, columns=['str'])
df['num'] = list(range(0, len(str_list)))

In [ ]:
df.sort_values(by='str', key=lambda x: x.str.encode('gbk'), ignore_index=True) # type: ignore

## 向一个excel文件循环写入数据


In [ ]:
import pandas as pd

In [ ]:
with pd.ExcelWriter('test.xlsx') as writer:
    for i in range(1, 4):
        df = pd.DataFrame(data=[i], columns=['num'])
        df.to_excel(writer, sheet_name=f'sheet{i}', index=False) # type: ignore

## 动态变量名

### `globals`方法


In [ ]:
for x in range(0, 7):
    globals()[f"variable{x}"] = f"Hello the variable number {x}!"

print(variable5)

### `locals()`方法


In [ ]:
for x in range(0, 7):
    locals()[f"variable{x}"] = f"Hello the variable number {x}!"

print(variable5)

### 新`locals()`方法


In [ ]:
# 定义变量名的规律
variable_name_pattern = "variable_{}"

# 定义生成变量的数量
num_variables = 5

# 循环生成变量名
for i in range(num_variables):
  variable_name = variable_name_pattern.format(i+1)
  # 将生成的变量名用于给变量赋值
  locals()[variable_name] = i+10
  print(variable_name, locals()[variable_name])

## 采样工作脚本测试

### 职业卫生


In [ ]:
import io
import openpyxl
import pandas as pd
from nptyping import DataFrame
# from pandas.api.types import CategoricalDtype
from my_modules.occupational_health import OccupationalHealthItemInfo, refresh_engaged_num
# from my_modules.occupational_health import SingleDayOccupationalHealthItemInfo

In [ ]:
company_name: str = 'MSCN'
project_name: str = '23ZXP0000'
# working_days: float = 5.0

file_path: str = r'./templates/项目信息试验模板.xlsx'
point_info_df: DataFrame = pd.read_excel(file_path, sheet_name='定点') # type: ignore
personnel_info_df: DataFrame = pd.read_excel(file_path, sheet_name='个体') # type: ignore

In [ ]:
new_project = OccupationalHealthItemInfo(company_name, project_name, point_info_df, personnel_info_df)

In [ ]:
new_project.point_info_df.head()

In [ ]:
new_project.point_info_df.query('采样点编号 == 25')#.tail()  # type: ignore

In [ ]:
new_project.personnel_info_df.head()

In [ ]:
new_project.point_deleterious_substance_df.head()

In [ ]:
new_project.personnel_deleterious_substance_df.head()

In [ ]:
new_project.factor_reference_df.head()

In [ ]:
file_io = new_project.get_dfs_num(['空白', '定点', '个体'])

with open('C:/Users/15210/Desktop/test1.xlsx', 'wb') as f:
    f.write(file_io.getvalue())

In [ ]:
engaged_num = 0
test_blank = new_project.get_single_day_blank_df(engaged_num, 1)
engaged_num = refresh_engaged_num(test_blank, '空白', engaged_num)
test_blank.head()

In [ ]:
test_point = new_project.get_single_day_point_df(engaged_num, 1)
test_point = pd.merge(test_point, test_blank, on='标识检测因素', how='left').fillna(0)
engaged_num = refresh_engaged_num(test_point, '定点', engaged_num)
test_point.head()

In [ ]:
test_personnel = new_project.get_single_day_personnel_df(engaged_num, 1)
engaged_num = refresh_engaged_num(test_personnel, '个体', engaged_num)
test_personnel.head()

In [ ]:
pd.merge(test_point, current_point_df, how='left', on='标识检测因素').fillna(0)

In [ ]:
p_point = pd.pivot_table(
    test_point,
    # index=['检测因素', '空白编号'],
    index=['检测因素'],
    # aggfunc={'起始编号': min, '终止编号': max}
    aggfunc={'空白编号': max, '起始编号': min, '终止编号': max}
)

p_point.head()

In [ ]:
p_point['定点数量'] = p_point['终止编号'] - p_point['起始编号'] + 1

p_point

In [ ]:
p_personnel = pd.pivot_table(
    test_personnel,
    index='检测因素',
    values='个体编号',
    aggfunc=[min, max]
).stack()

p_personnel

In [ ]:
p_personnel = p_personnel.reset_index().set_index('检测因素').drop('level_1', axis=1).rename(columns={'min': '个体起始编号', 'max': '个体终止编号'})

p_personnel

In [ ]:
pd.concat(
    [p_point, p_personnel],
    axis=1
).fillna(0).applymap(int)